# 🚀 FreqAI Backtest با GPU - Google Colab

این notebook برای اجرای backtest استراتژی FreqAI با GPU است.

**⚠️ قبل از شروع:**
1. Runtime → Change runtime type → Hardware accelerator → **T4 GPU**
2. Save

**زمان تخمینی:** 30-60 دقیقه (با GPU)

## 1️⃣ نصب Dependencies

In [ ]:
# نصب TA-Lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -q
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr > /dev/null 2>&1
!make > /dev/null 2>&1
!make install > /dev/null 2>&1
%cd ..

print("✅ TA-Lib نصب شد")

In [ ]:
# نصب Freqtrade و dependencies
!pip install freqtrade[freqai] -q
!pip install technical pandas-ta -q

print("✅ Freqtrade نصب شد")

## 2️⃣ بررسی GPU

In [ ]:
# بررسی GPU
!nvidia-smi

import torch
print(f"\n✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU not available! Change runtime type to GPU.")

## 3️⃣ Clone Project از GitHub

In [ ]:
# Clone project
!git clone https://github.com/aminak58/freqai-futures-strategy.git
%cd freqai-futures-strategy

print("✅ Project cloned")

## 4️⃣ دانلود Historical Data

⚠️ **نکته:** اگر در ایران هستید و VPN ندارید، می‌توانید data را از سیستم محلی upload کنید (سلول بعدی)

In [ ]:
# دانلود 1 سال data
!freqtrade download-data \
  --exchange binance \
  --pairs BTC/USDT:USDT ETH/USDT:USDT SOL/USDT:USDT \
  --timeframes 5m 15m 1h \
  --days 365 \
  --trading-mode futures \
  --config config/config.json

print("✅ Data دانلود شد")

### (اختیاری) Upload Data از سیستم محلی

In [ ]:
# اگر می‌خواهید data را از سیستم محلی upload کنید:
# 1. ابتدا در سیستم محلی data را zip کنید:
#    zip -r user_data.zip user_data/data/
# 2. سپس این cell را run کنید:

from google.colab import files
uploaded = files.upload()

!unzip user_data.zip -d .
print("✅ Data uploaded")

## 5️⃣ اجرای Backtest

این مرحله 30-60 دقیقه طول می‌کشد.

In [ ]:
# اجرای backtest کامل
!freqtrade backtesting \
  --strategy FreqAIHybridStrategy \
  --config config/config.json \
  --freqaimodel LightGBMRegressorMultiTarget \
  --timerange 20250101-20251012 \
  --export trades

print("\n🎉 Backtest تمام شد!")

### (اختیاری) Backtest سریع برای Test

اگر می‌خواهید سریع test کنید، فقط 1 pair و 3 ماه:

In [ ]:
# Quick test (10-15 دقیقه)
!freqtrade backtesting \
  --strategy FreqAIHybridStrategy \
  --config config/config.json \
  --freqaimodel LightGBMRegressorMultiTarget \
  --timerange 20250701-20251012 \
  --pairs BTC/USDT:USDT \
  --export trades

print("\n✅ Quick test تمام شد!")

## 6️⃣ مشاهده نتایج

In [ ]:
# نمایش خلاصه نتایج
import json
import glob

# پیدا کردن آخرین backtest result
result_files = glob.glob('user_data/backtest_results/backtest-result*.json')
latest_result = max(result_files, key=lambda x: x)

with open(latest_result, 'r') as f:
    results = json.load(f)

print("📊 نتایج Backtest:")
print("="*50)
strategy_data = results['strategy']['FreqAIHybridStrategy']
print(f"Total Profit: {strategy_data['profit_total']:.2%}")
print(f"Sharpe Ratio: {strategy_data.get('sharpe', 'N/A')}")
print(f"Max Drawdown: {strategy_data['max_drawdown']:.2%}")
print(f"Win Rate: {strategy_data['wins'] / strategy_data['trades']:.2%}")
print(f"Total Trades: {strategy_data['trades']}")
print(f"Avg Trade Duration: {strategy_data['avg_duration']}")
print("="*50)

## 7️⃣ Download نتایج

In [ ]:
# Download backtest results
from google.colab import files

# فایل JSON نتایج
files.download(latest_result)

print("📥 فایل نتایج download شد")

In [ ]:
# Download trained models (برای استفاده در live trading)
!zip -r models.zip user_data/models/
files.download('models.zip')

print("📥 Models download شد")

## 8️⃣ Monitoring (اختیاری)

برای مشاهده real-time progress:

In [ ]:
# نمایش logs
!tail -f user_data/logs/freqtrade.log

## 9️⃣ Cleanup (اختیاری)

برای آزاد کردن فضا:

In [ ]:
# پاک کردن فایل‌های موقت
!rm -rf ta-lib*
!rm -rf user_data/data/  # بعد از download models

print("✅ Cleanup انجام شد")